In [13]:
import pandas as pd
import os

In [14]:
## GOOGLE DRIVE
from google.oauth2 import service_account
from googleapiclient.discovery import build

#DOWNLOAD ARQUIVOS
import io
from googleapiclient.http import MediaIoBaseDownload

In [15]:
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]

credentials = service_account.Credentials.from_service_account_file(
    r'E:\Documentos\python\.gcredentials\conectar-integracao\conectar-integracao-4748610d1d6e.json', scopes=scope)

service = build('drive', 'v3', credentials=credentials)

#Listar aquivos na Pasta
topFolderId = '1hMXHfkW9AOg4WFEXYLdpgyIiNFs3NCXQ' 

items = []
pageToken = ""
while pageToken is not None:
    response = service.files().list(q="'" + topFolderId + "' in parents", pageSize=1000, pageToken=pageToken, fields="nextPageToken, files(id, name)").execute()
    items.extend(response.get('files', []))
    pageToken = response.get('nextPageToken')

In [16]:
items

[{'id': '14fw_Hd4KAlMlvs-MsXPJP-2CCjkSiwqL',
  'name': '3165008 - São Tiago MG.xlsx'},
 {'id': '15wl9I4ozCI4iGKyaprq2mky9SQ3_nbVK',
  'name': '3303906 - Petrópolis RJ.xlsx'},
 {'id': '1XQ49-nq3zRSEvNXLt8YXbhCj1Klgbw9B',
  'name': '4106902 - Curitiba PR.xlsx'},
 {'id': '1KKJnh-WiIJr1-e8ZeD3w_C-tdP_y0bKF',
  'name': '4121208 - Quitandinha PR.xlsx'},
 {'id': '12vmNRjDX24q0hwA2WNU9GB19IQKREXqD',
  'name': '2111300 - São Luís MA.xlsx'},
 {'id': '1LGk1eb_gRnoTlK7opXUu63gXFaY62r_2',
  'name': '3120201 - Cristais MG.xlsx'},
 {'id': '1zWcd5ru9gJp1EvRTvcMRm1kC3_VQcwSK',
  'name': '4113304 - Laranjeiras do Sul PR.xlsx'},
 {'id': '1Y3dAjeqc0R9EV_Ps8FVbfmfV83P3OS8H',
  'name': '3501905 - Amparo SP.xlsx'},
 {'id': '1SMq8vboeygg1lReqPs4XQWy5gaDpEvcL',
  'name': '2604106 - Caruaru PE.xlsx'},
 {'id': '1molLmiS6oAJAH-QHfXPpVwTTofl32Ggi',
  'name': '2921005 - Mata de São João BA.xlsx'},
 {'id': '1YaAMnzRVEjEEgFIVD9YynF_54_bcTWHC',
  'name': '4112603 - Jardim Olinda PR.xlsx'},
 {'id': '1t2sRNVbfBSVsswtgu0

In [17]:
#DF com o ID, Nome do Arquivo e COD MUN
file_id = []
nome = []

for f in range(0, len(items)):
    file_id.append(items[f].get('id'))
    nome.append(items[f].get('name'))


arquivo = pd.DataFrame(list(zip(file_id, nome)), columns=['File ID', 'Nome'])
arquivo['COD MUN'] = arquivo['Nome'].str.split('-', expand=True)[0]

In [18]:
def baixar_arquivo(file_id, nome):
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(r'E:\Documentos\python\fluxo_pedidos\programacao\{0}'.format(nome), 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        #print("Download %d%%." % int(status.progress() * 100))
        print(nome)

In [19]:
#Baixar os arquivos na pasta
print('Rodando script 01_programacao')
for index, row in arquivo.iterrows():
    file_id = row['File ID']
    nome = row['Nome']
    baixar_arquivo(file_id, nome)

Rodando script 01_programacao
3165008 - São Tiago MG.xlsx
3303906 - Petrópolis RJ.xlsx
4106902 - Curitiba PR.xlsx
4121208 - Quitandinha PR.xlsx
2111300 - São Luís MA.xlsx
3120201 - Cristais MG.xlsx
4113304 - Laranjeiras do Sul PR.xlsx
3501905 - Amparo SP.xlsx
2604106 - Caruaru PE.xlsx
2921005 - Mata de São João BA.xlsx
4112603 - Jardim Olinda PR.xlsx
4109401 - Guarapuava PR.xlsx
5203906 - Buriti Alegre GO.xlsx
3532207 - Narandiba SP.xlsx
3111200 - Campo Belo MG.xlsx
3500303 - Aguaí SP.xlsx
3543402 - Ribeirão Preto SP.xlsx
4109609 - Guaratuba PR.xlsx
4205407 - Florianópolis SC.xlsx
3156908 - Sacramento MG.xls.xlsx
4218707 - Tubarão SC.xlsx


In [20]:
def abrir_arquivo(nome_arquivo):
    COD_MUN = nome_arquivo[47:54]
    
    base = pd.read_excel(nome_arquivo, usecols='B:O', skiprows=6, nrows=8 )
    
    base = base.iloc[:,[0,11,12]]
    base.loc[:,'COD MUN'] = COD_MUN
    base_mun = base.melt(id_vars=['Nº', 'COD MUN'], var_name='COTA', value_name='QUANTIDADE')
    base_mun['COTA'] = base_mun['COTA'].str.replace('QUANT. COTA ', "")
    return base_mun

In [21]:
# Listar arquivos do diretorio, abrir o arquivo e consolidar em um DF
diretorio = r'E:\Documentos\python\fluxo_pedidos\programacao'

base_aux = pd.DataFrame()

for arquivo in os.listdir(diretorio):
    filename = os.fsdecode(arquivo)
    nome_arquivo = diretorio + '\\' + filename
    
    resultado = abrir_arquivo(nome_arquivo)
        
    base_aux = pd.concat([base_aux, resultado])

base_aux['COD MUN'] = base_aux['COD MUN'].astype(str)
base_aux['ETAPA'] = 'Programação'

In [22]:
base_aux = base_aux[base_aux['QUANTIDADE'] > 0]

In [23]:
base_aux

,Nº,COD MUN,COTA,QUANTIDADE,ETAPA
0,1,2111300,AMPLIADA,202500.0,Programação
1,2,2111300,AMPLIADA,405000.0,Programação
2,3,2111300,AMPLIADA,405000.0,Programação
3,4,2111300,AMPLIADA,112500.0,Programação
4,5,2111300,AMPLIADA,24750.0,Programação
...,...,...,...,...,...
11,4,5203906,RESERVADA,600.0,Programação
12,5,5203906,RESERVADA,180.0,Programação
13,6,5203906,RESERVADA,180.0,Programação
14,7,5203906,RESERVADA,60.0,Programação


In [24]:
base_aux.to_excel(r'E:\Documentos\python\fluxo_pedidos\bases\base_programacao.xlsx', index=False)